<a href="https://colab.research.google.com/github/fothstatshs-design/PROJECTS/blob/main/Flight_Delay_Statistics_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================================================================
# Flight Delay Statistics Dashboard
# Optimized to run within Google Colab using modern Dash syntax.
# =========================================================================

!pip install dash pandas plotly
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
print("Loading flight data...")
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv',
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str,
                                   'Div2Airport': str, 'Div2TailNum': str})
print("Data loaded successfully.")

# Create a standard Dash application instance.
app = dash.Dash(__name__)

# Suppress callback exceptions is crucial for multi-output dashboards
app.config.suppress_callback_exceptions = True

# Build dash app layout
app.layout = html.Div(children=[
    html.H1(
        'Flight Delay Time Statistics',
        style={
            'textAlign': 'center',
            'color': '#503D36',
            'fontSize': 30
        }
    ),
    html.Div([
        html.Label("Input Year: ", style={'fontSize': 30}),
        dcc.Input(
            id='input-year',
            value='2010',
            type='number',
            min=2005,
            max=2020,
            style={
                'height':'35px',
                'fontSize': 30,
                'textAlign': 'center'
            }
        ),
    ], style={'textAlign': 'center', 'margin': '20px 0', 'fontSize': 30}),

    html.Br(),
    html.Br(),

    # Segment 1: Carrier and Weather Delay
    html.Div([
        html.Div(dcc.Graph(id='carrier-plot'), style={'width': '50%'}),
        html.Div(dcc.Graph(id='weather-plot'), style={'width': '50%'})
    ], style={'display': 'flex'}),

    # Segment 2: NAS and Security Delay
    html.Div([
        html.Div(dcc.Graph(id='nas-plot'), style={'width': '50%'}),
        html.Div(dcc.Graph(id='security-plot'), style={'width': '50%'})
    ], style={'display': 'flex'}),

    # Segment 3: Late Aircraft Delay (wider graph)
    html.Div(dcc.Graph(id='late-plot'), style={'width':'100%'})
])

""" Compute_info function description (Helper function)
Performs computation for creating charts and plots.
"""
def compute_info(airline_data, entered_year):
    # Select data for the entered year
    df =  airline_data[airline_data['Year']==int(entered_year)]

    # Compute delay averages grouped by Month and Airline
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

"""Callback Function
Function that returns figures using the provided input year.
"""
# Callback decorator
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))

# Computation to callback function and return graph
def get_graph(entered_year):

    # Compute required information for creating graphs
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)

    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average Carrier Delay (minutes) by Airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average Weather Delay (minutes) by Airline')
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS Delay (minutes) by Airline')
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average Security Delay (minutes) by Airline')
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average Late Aircraft Delay (minutes) by Airline')

    # Ensure titles are readable
    for fig in [carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]:
        fig.update_layout(
            title_font_size=14,
            legend_font_size=10,
            margin=dict(l=20, r=20, t=40, b=20)
        )

    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

# Run the app
if __name__ == '__main__':
    # Use the modern app.run() which is optimized for Colab environments
    app.run(host='0.0.0.0', port=8050)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 78.4 MB/s eta 0:00:00
Loading flight data...
Data loaded successfully.


<IPython.core.display.Javascript object>